# 03 Nettoyage population

## Objectifs

1. Mise en place des données de recensement 2018 et 2022
2. Aggrégation au niveau des départements
3. Interpolation linéaire par mois

In [2]:
import pandas as pd
import numpy as np

In [9]:
path = "../data_raw/estim-pop-dep-sexe-gca-1975-2022.xls"

In [53]:
dfs = []

for i in range(2017, 2023):
    
    df = pd.read_excel(
        path,
        sheet_name=str(i),
        skiprows=4,
        usecols=range(8),
        names=[
            "code",
            "departement",
            "nb_00_19",
            "nb_20_39",
            "nb_40_59",
            "nb_60_74",
            "nb_75_plus",
            f"{i}"

        ]
    )

    df["departement"] = df["departement"].str.replace("-", " ", regex=False).str.lower().str.strip()

    df["departement"] = df["departement"].str.replace("*", "", regex=False)
    
    # df["month"] = pd.to_datetime(f"01/01/{i}")
    
    dfs.append(df[["code", "departement", f"{i}"]])
    
df.head()

,code,departement,nb_00_19,nb_20_39,nb_40_59,nb_60_74,nb_75_plus,2022
0,01,ain,169605.0,149821.0,181185.0,107629.0,57151.0,665391.0
1,02,aisne,127886.0,113607.0,132928.0,99375.0,50607.0,524403.0
2,03,allier,66254.0,62020.0,83598.0,72268.0,47617.0,331757.0
3,04,alpes de haute provence,33620.0,30724.0,42956.0,36032.0,22250.0,165582.0
4,05,hautes alpes,29263.0,26865.0,37157.0,30480.0,17294.0,141059.0


In [54]:
df = dfs[0]

for df_temp in dfs[1:]:
    df = pd.merge(
        left=df,
        right=df_temp,
        how="outer",
        on=["code", "departement"]
    )
    
df.isna().sum()

code           1
departement    6
2017           3
2018           3
2019           3
2020           3
2021           3
2022           3
dtype: int64

In [55]:
df[df["departement"].isna()]

,code,departement,2017,2018,2019,2020,2021,2022
96,France métropolitaine,NaN,64639133.0,64844037.0,65096768.0,65284389.0,65447454.0,65627454.0
102,DOM,NaN,2135349.0,2148122.0,2161214.0,2169733.0,2178942.0,2185942.0
103,France métropolitaine et DOM,NaN,66774482.0,66992159.0,67257982.0,67454122.0,67626396.0,67813396.0
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Source : Insee - Estimations de population,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,Source : Insee - Estimations de population (ré...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df.tail()

,code,departement,2017,2018,2019,2020,2021,2022
102,DOM,NaN,2135349.0,2148122.0,2161214.0,2169733.0,2178942.0,2185942.0
103,France métropolitaine et DOM,NaN,66774482.0,66992159.0,67257982.0,67454122.0,67626396.0,67813396.0
104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Source : Insee - Estimations de population,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,Source : Insee - Estimations de population (ré...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df = df.dropna()

In [58]:
df.tail()

,code,departement,2017,2018,2019,2020,2021,2022
97,971,guadeloupe,390253.0,387629.0,384239.0,380905.0,377856.0,372939.0
98,972,martinique,372594.0,368783.0,364508.0,360280.0,356029.0,350373.0
99,973,guyane,268700.0,276128.0,281678.0,286032.0,290528.0,294436.0
100,974,la réunion,853659.0,855961.0,861210.0,863197.0,866181.0,868846.0
101,976,mayotte,250143.0,259621.0,269579.0,279319.0,288348.0,299348.0


In [59]:
df = df.melt(
    id_vars=["code", "departement"],
    value_vars=df.columns[2:],
    value_name="population"
)
df.head()

,code,departement,variable,population
0,01,ain,2017,643350.0
1,02,aisne,2017,534490.0
2,03,allier,2017,337988.0
3,04,alpes de haute provence,2017,163915.0
4,05,hautes alpes,2017,141284.0


In [60]:
df = df.rename(columns={"variable": "year"})
df.head()

,code,departement,year,population
0,01,ain,2017,643350.0
1,02,aisne,2017,534490.0
2,03,allier,2017,337988.0
3,04,alpes de haute provence,2017,163915.0
4,05,hautes alpes,2017,141284.0


In [61]:
df.to_csv("../data_clean/clean_population.csv", index=False)